In [1]:
import pandas as pd
import bs4
import cloudscraper

In [2]:
#Substituir o caminho do arquivo para o caminho do arquivo que você baixou
df = pd.read_excel('../Base/acoes-economatica.xlsm', sheet_name='Método Davi')
scrapper = cloudscraper.create_scraper()

In [3]:
tickers = df["Unnamed: 1"].dropna().values
tickers = tickers[2:].tolist()

In [5]:
def get_acao(ticker):
    tag_along = '--'
    try: 
        url = f'https://statusinvest.com.br/acoes/{ticker}'
        response = scrapper.get(url)
        soup = bs4.BeautifulSoup(response.text, 'html.parser')
        infos = soup.find_all('div', class_='info')
        tags = []
        for info in infos:
            if 'Tag Along' in info.text:
                tags = info.find_all('strong')
                
        tag_along = tags[0].text if tags else '--'
    except:
        tag_along = 'Erro'
    
    return tag_along

In [ ]:
tag_alongs = {}
for ticket in tickers:
    tag_along = get_acao(ticket)
    print(f'{ticket}: {tag_along}')
    tag_alongs[ticket] = tag_along

In [12]:
copy = tag_alongs
for key, value in copy.items():
    if value != '--':
        value = value.replace(' ', '')
    else:
        value = 0
        
df['Tag Along'] = df['Ticket'].map(tag_alongs)
df.to_csv('tag_along.csv', sep=';', encoding='utf-8', index=False)